In [1]:
import math
import matplotlib as mpl
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
import json
import base64
import os
import sys

# Smoothed Armchair Graphene Nanoribbon Junctions Explorer

As Moore's law approaches its fundamental limits, the development of nanoelectronic devices using low-dimension materials has become a promising avenue for further miniaturization and performance improvements. Among the various novel materials, graphene nanoribbons (GNRs) have emerged as particularly attractive candidates due to their unique electronic properties. However, the design of efficient nanoelectronic components with a minimal spatial footprint remains a significant challenge. This tool allows to explore a wide dataset of GNR junctions and investigate various strategies for designing optimal-sized interconnects. For each set of parameters, the tool will provide various informations about the junction's electronic properties. All the raw data can then be downloaded for further analysis.

If you use this tool, please cite the following works:

* **J. Leuenberger, K. Cernevics, O. V. Yazyev, Optimizing Nanoelectronics Footprint by Rounding Graphene Nanoribbon Junctions**
* **J. Leuenberger, K. Cernevics, O. V. Yazyev, Smoothed Armchair Graphene Nanoribbon Junctions Explorer (SAJEx), doi:10.tbd/tbd**

In [2]:
path = "./data" if os.path.isdir("./data") else "/data"
descriptions_width = '150px'
contents_width = '450px'
buttons_width = '450px'

In [3]:
def get_wext_max(N_value):
    security = 1
    return math.floor(N_value / 2) - 1 - security

def get_N_min(wext_value):
    for N in range(20, 80+1, 3):
        if get_wext_max(N) >= wext_value:
            return N

In [4]:
shape = widgets.Dropdown(options=['armchair', 'round', 'zigzag'], value='armchair', description='shape', style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
N = widgets.IntSlider(min=20, max=80, step=3, description="N", style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
wext = widgets.IntSlider(min=0, max=get_wext_max(N.value), step=1, description="wₑₓₜ", style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
a_ratio = widgets.Text(value="", description='Armchair ratio', disabled=False, style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
z_ratio = widgets.Text(value="", description='Zigzag ratio', disabled=False, style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
tau_around_0p00_ev = widgets.Text(value="", description='τ @ 0.00 eV', disabled=False, style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
tau_around_0p01_ev = widgets.Text(value="", description='τ @ 0.01 eV', disabled=False, style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))
tau_around_0p10_ev = widgets.Text(value="", description='τ @ 0.10 eV', disabled=False, style={'description_width': descriptions_width}, layout=widgets.Layout(width=contents_width))

html_button_summary = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""", layout=widgets.Layout(width=buttons_width))

html_button_conductance = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""", layout=widgets.Layout(width=buttons_width))

html_button_vertices = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""", layout=widgets.Layout(width=buttons_width))


html_button_edges = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""", layout=widgets.Layout(width=buttons_width))



html_images = widgets.HTML(
value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<table>
<tr>
<td>
<img src="data:image/png;base64,IMAGE1" width="100%">
</td>
<td>
<img src="data:image/png;base64,IMAGE2" width="100%">
</td>
<td>
<img src="data:image/png;base64,IMAGE3" width="100%">
</td>
<td>
<img src="data:image/png;base64,IMAGE4" width="100%">
</td>
</tr>
</table>
</body>
</html>""")



In [5]:
def update_image(shape, N, r):
    # TODO : rework this function or remove it if not necessary
    return
    fig, axs = plt.subplots(ncols=4, nrows=1, figsize=(19.2, 4.8), dpi=100)
    try:
        axs[0].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/edge.png"))
        axs[1].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/conductance.png"))
        axs[2].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/ldos.png"))
        axs[3].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/current.png"))
        for ax in axs.ravel():
            ax.axis("off")
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_anchor('S')
        plt.subplots_adjust(wspace=-0.05, hspace=0)
    except Exception as e:
        print(e)
        # plt.close()
        # print(f"Missing data !")
        pass
    
def update_wext_max(*args):
    wext_current = wext.value
    wext_max = get_wext_max(N.value)
    if wext_current > wext_max:
        wext.value = wext_max
    wext.max = wext_max
    
def update_N_min(*args):
    N_current = N.value
    N_min = get_N_min(wext.value)
    if N_current < N_min:
        N.value = N_min
    N.min = N_min

def update_texts(change):
    try:
        infos = json.load(open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/summary.json"))
        a_ratio.value = f'{infos["armchair_ratio"]:.1%}'
        z_ratio.value = f'{infos["zigzag_ratio"]:.1%}'
        tau_around_0p00_ev.value = f'{infos["tau_around_0p00_ev"]:.1%}'
        tau_around_0p01_ev.value = f'{infos["tau_around_0p01_ev"]:.1%}'
        tau_around_0p10_ev.value = f'{infos["tau_around_0p10_ev"]:.1%}'
    except Exception as e:
        print(e)
        pass

def update_downloadbutton(*args):
    
    base_html = """<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="FILENAME" href="data:text/json;base64,PAYLOAD" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button" style="width: 100%;">TEXT</button>
    </a>
    </body>
    </html>"""

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/summary.json").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"summary_shape={shape.value}_N={N.value}_wext={wext.value}.json"
    html_button_summary.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Summary (JSON, ~{sys.getsizeof(payload)/1024:.1f} kB)")

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/conductance.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"conductance_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_conductance.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Conductance (CSV, ~{sys.getsizeof(payload)/1024:.1f} kB)")

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/vertices.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"vertices_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_vertices.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Vertices (CSV, ~{sys.getsizeof(payload)/1024:.1f} kB)")

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/edges.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"edges_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_edges.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Edges (CSV, ~{sys.getsizeof(payload)/1024:.1f} kB)")


def update_images(*args):
    base_html = """<html>
    <head>
    <meta name="viewport" content="width=100%; initial-scale=1;">
    </head>
    <body>
    <table>
    <tr>
    <td>
    <img src="data:image/png;base64,IMAGE1" width="100%">
    </td>
    <td>
    <img src="data:image/png;base64,IMAGE2" width="100%">
    </td>
    <td>
    <img src="data:image/png;base64,IMAGE3" width="100%">
    </td>
    <td>
    <img src="data:image/png;base64,IMAGE4" width="100%">
    </td>
    </tr>
    </table>
    </body>
    </html>"""

    img1 = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/edge.png", "rb").read()
    img1 = base64.b64encode(img1).decode().replace("\"", "`")
    img2 = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/conductance.png", "rb").read()
    img2 = base64.b64encode(img2).decode().replace("\"", "`")
    img3 = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/ldos.png", "rb").read()
    img3 = base64.b64encode(img3).decode().replace("\"", "`")
    img4 = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/current.png", "rb").read()
    img4 = base64.b64encode(img4).decode().replace("\"", "`")

    html_images.value = base_html.replace("IMAGE1", img1).replace("IMAGE2", img2).replace("IMAGE3", img3).replace("IMAGE4", img4)
    

## Inputs

In order to investigate the strategies for designing a GNR junction, the following parameters can be chosen:

* **shape** corresponds to the junction's tip as material is removed. This can either be "armchair" which maximises the amount of armchair edges, "zigzag" which maximises the amount of zigzag edges, or "round" which is an in-between.
* **N** is the width of the GNRs, in number of atom layers. Allowed values are **N** $\in[5, 80]$. Note that only metallic GNRs are considered, i.e. **N** $=3n+2$.
* **wₑₓₜ** is the smoothening parameter and quantifies the amount of material removed. The larger **wₑₓₜ**, the more predominant **shape** will be.

In [6]:
# 'update_texts'
shape.observe(update_texts, names='value')
N.observe(update_texts, names='value')
wext.observe(update_texts, names='value')

# 'update_wext_max'
N.observe(update_wext_max, names='value')
wext.observe(update_N_min, names='value')

# 'update_downloadbutton'
shape.observe(update_downloadbutton, names='value')
N.observe(update_downloadbutton, names='value')
wext.observe(update_downloadbutton, names='value')

# 'update_images'
shape.observe(update_images, names='value')
N.observe(update_images, names='value')
wext.observe(update_images, names='value')

# initialize all widgets
update_texts(None)
update_downloadbutton(None)
update_wext_max(None)
update_images(None)

# display sliders
display(widgets.interactive(update_image, shape=shape, N=N, r=wext))

interactive(children=(Dropdown(description='shape', layout=Layout(width='450px'), options=('armchair', 'round'…

## Results

The tool does provide the following graphical results:
- **The junction's shape**, where the edges are highlighted in red or blue depending on if they are classified as "armchair vertices" or "zigzag vertices", respectively.
- **The transmission plot**, where the transmission of the junction (red solid line) is plotted as a function of the energy. As a comparison, the transmission of the corresponding straight GNR (black dashed line) is also plotted.
- **The local density of states (LDOS) plot**, where the circles' colors depend on the sublattice on which they are.
- **The probability density current plot**. Note that for readability, only flows greater or equal to one tenth of the maximum flow are shown.

In [7]:
display(widgets.HBox([html_images]))

It does also provide numerical results. Below you will find the following:

- **Armchair ratio** is the ratio of armchair vertices over the total number of vertices which compose the edge.
- **Zigzag ratio** is the ratio of zigzag vertices over the total number of vertices which compose the edge.
- **$\tau$ @ 0.00 eV** is the preserved conductance of the junction at exactly (up to numerical errors) 0.00 eV.
- **$\tau$ @ 0.01 eV** is the preserved conductance of the junction averaged over a 0.01 eV window around 0.00 eV.
- **$\tau$ @ 0.10 eV** is the preserved conductance of the junction averaged over a 0.10 eV window around 0.00 eV.

$\tau$ is the preserved conductance at or around a given energy, given by
$$
\tau(E)=\frac{\int_{E-\frac{1}{2}\delta E}^{E+\frac{1}{2}\delta E}G(E')dE'}{\int_{E-\frac{1}{2}\delta E}^{E+\frac{1}{2}\delta E}G_\mathrm{lead}(E')dE'},
$$
where $\delta E$ is the energy window, written here as "$\tau$ @ $\delta E$".

Note that more detailed results can be downloaded below.

In [8]:
display(
    widgets.VBox([
        a_ratio,
        z_ratio,
        tau_around_0p00_ev,
        tau_around_0p01_ev,
        tau_around_0p10_ev,
    ]),
)

## Download

Here you can download more detailed informations about the currently selected junction. The files and the informations they contain are described below.

**Summary** contains the following informations:
- `shape`: a string either "armchair", "zigzag" or "round".
- `N`: an integer between 20 and 80, corresponding to the width of the GNRs.
- `wext`: an integer corresponding to the smoothening parameter.
- `tau_around_0p00_ev`: a float corresponding to the preserved conductance in [$G_0$] at 0.00 eV.
- `tau_around_0p01_ev`: a float corresponding to the preserved conductance in [$G_0$] averaged within a 0.01 eV window around 0.00 eV.
- `tau_around_0p10_ev`: a float corresponding to the preserved conductance in [$G_0$] averaged within a 0.10 eV window around 0.00 eV.
- `edge_length`: an integer corresponding to the total length of the junction's edges.
- `edge_sequence`: a string made of "A"s, "Z"s and "X"s, corresponding to the edge structure.
- `armchair_ratio`: a float corresponding to the ratio of armchair edges in the edge.
- `zigzag_ratio`: a float corresponding to the ratio of zigzag edges in the edge.
- `other_ratio`: a float corresponding to the ratio of other types of edges in the edge.

**Conductance** contains the following informations:
- `energy`: a list of floats between -0.1 and 0.1, corresponding to energies in [$\mathrm{eV}$].
- `conductance_lead`: a list of floats, corresponding to the conductance in [$G_0$] of the leads at the corresponding energies.
- `conductance_junction`: a list of floats, corresponding to the conductance in [$G_0$] of the junction at the corresponding energies.

**Vertices** contains the following informations:
- `vertex_index`: a list of integers, corresponding to the indices of the vertices. Note that these indices are related to the ones in the file **Edges**.
- `vertex_x`: a list of floats, corresponding to the $x$ positions of the vertices in [$\sqrt{3}a$].
- `vertex_y`: a list of floats, corresponding to the $y$ positions of the vertices in [$a$].
- `ldos`: a list of floats, corresponding to the local density of states at the given vertices.

**Edges** contains the following informations:
- `edge_index`: a list of integers, corresponding to the indices of the edges
- `vertex_1_index`: a list of integers, corresponding to the indices of the first vertex of the edges
- `vertex_2_index`: a list of integers, corresponding to the indices of the second vertex of the edges
- `current`: a list of floats, corresponding to the probability current in [$G_0$] flowing through the edges. Note that if the current is negative, it means that the current is flowing from vertex 2 to vertex 1.

In [9]:
display(
    widgets.VBox([
        html_button_summary,
        html_button_conductance,
        html_button_vertices,
        html_button_edges,
    ]))